# Dealing with unclean data

We're going to look at data that may require some cleansing.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


## Read the admissions data that is not so clean

In [ ]:
data_location  = 'https://elephantscale-public.s3.amazonaws.com/data/college-admissions/admission-data-dirty.csv'

admissions = pd.read_csv(data_location)
print("admissions size : ", admissions.size)
admissions

## Get Summary
See what we get.  It will skip null values

In [ ]:
## TODO : use 'describe' functions 
admissions.describe()

In [ ]:
## TODO : make describe include all columns
admissions.describe(include = 'all')

In [ ]:
## TODO : Describe more than one column : gre and gpa
## Hint : add 'gpa' column
admissions[['gre', 'gpa']].describe()

## Drop all null values

In [ ]:
print("raw data size : ", admissions.size)

## TODO : use 'dropna' function
dropped_na = admissions.dropna()
print()
print("after drop size : ", dropped_na.size)
dropped_na


In [ ]:
# only drop nulls from admit & gre column
print("raw data size : ", admissions.size)

print()

dropped2 = admissions.dropna(subset=['admit', 'gre'])
print("after drop size : ", dropped2.size)
dropped2

## Fill in the values

In [ ]:
## TODO :  fill every thing with zero
## Hint : use 'fillna'
zero_fill = admissions.fillna(0)
zero_fill

In [ ]:
# or we can specify per column default value
## TODO : specify different default values per column
##        default value for gre = -100
fill2 = admissions.fillna({'admit': -1, 'gre': -100, 'gpa':-1, 'rank':10})
fill2

## Replace values

In [ ]:
print (admissions)

admissions2 = admissions.copy(deep=True)

## TODO : use replace to change 800 to 1000
## Hint : replace (800, 1000)
admissions2['gre'].replace(800, 1000, inplace=True)

print()
print (admissions2)

## Clean out RANK column

In [ ]:
## TODO : filter out any thing other than 1,2,3,4  in rank column
a = admissions[admissions['rank'].isin(['1','2','3','4'])]
a

#Exercise 2  - Cleaning up House Sales Data (★★☆)

##Read the house-sales-simplified.csv.
> *See the shape , describe, info functions of dataframe*

> *Find Numerical and Categorical columns in the data*

In [ ]:
data_file_path  = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-simplified.csv'
df_house = pd.read_csv(data_file_path)

#print ( "\nHouse DataFrame size: {0}\n".format(df_house.size))
print ( "\nHouse DataFrame shape: {0}\n".format(df_house.shape))
##print ( "\nHouse DataFrame info: {0}\n".format(df_house.info()))
pd.set_option('display.float_format', lambda x: '%.5f' % x)

numerical_Columns = df_house.select_dtypes([np.number]).columns.tolist()
categorical_Columns = df_house.select_dtypes( exclude = [np.number] ).columns.tolist()

numerical_Columns.remove('ID')
numerical_Columns.remove('YrBuilt')
numerical_Columns.remove('Bedrooms')

categorical_Columns.insert(0,'Bedrooms')
categorical_Columns.remove("Date")
print ("\nnumerical_Columns: {0} \ncategorical_Columns: {1} \n".format(numerical_Columns,categorical_Columns  ) )
df_house.describe(include='all')

##print ( 'Date colum type is ' ,  df_house.Date.dtype)



In [ ]:
## see the frequency of all numerical columns and outliers

sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})

## distribution of all numerica columns
##df_house[numerical_Columns].hist(bins=10, figsize=(10, 5), layout=(3, 3))
print ( "\n Distribution of all numerical values \n")
df_house[numerical_Columns].hist(bins=15, figsize=(15, 7), layout=(2, 3))

In [ ]:
print ( "\nDistribution of categorical values \n")
## distribution of categorical columns
listsize = len( categorical_Columns)
ncol = 3
if (  listsize < ncol ):
  ncol = listsize
nrow = round(len(categorical_Columns)/ ncol )

fig, axes_types = plt.subplots(nrows= nrow , ncols= ncol, sharex=False, sharey=True, squeeze=True, figsize=(8,4))

'''
for variable, subplot in zip(categorical_Columns, ax.flatten()):
      sns.countplot(df_house[variable], ax=subplot)
      for label in subplot.get_xticklabels():
        label.set_rotation(45)
'''

for i, ax in enumerate(fig.axes):
    if i <  listsize:
        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
        xlabel = categorical_Columns[i]
        sns.countplot(x= df_house[xlabel] , alpha=0.7, data=df_house, ax=ax)

fig.tight_layout()

In [ ]:
print ( "\nDistribution of SalePrice by categorical values \n")
## distribution of categorical columns
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=True, squeeze=True, figsize=(20,10))

for variable, subplot in zip(categorical_Columns, ax.flatten()):
      sns.boxplot(x=variable, y='SalePrice', data=df_house, ax=subplot)
      for label in subplot.get_xticklabels():
        label.set_rotation(90)

In [ ]:
print("\nFinding correlation between features \n")
# Finding the relations between the variables.
plt.figure(figsize=(20,10))
corr_values= df_house.corr()
sns.heatmap(corr_values,cmap="BrBG",annot=True)
##c

In [ ]:
## Identify columns that need data cleanup
## Hint : Zipcode
plt.figure(figsize=(9, 8))
'''
sns.distplot(
    df_house.ZipCode , bins= 10, color='g',norm_hist=False, kde=True, hist_kws={"alpha": 1}
).set(xlabel='ZipCode', ylabel='Count')
##plt.xlim(-10, df_house.ZipCode.max())
'''

##sns.violinplot(x=df_house['ZipCode'])
sns.violinplot(x="ZipCode", y= "PropertyType", data=df_house)
df_Zips =  df_house[ (df_house["ZipCode"]== df_house.ZipCode.min())] 
df_Zips.describe(include='all')

In [ ]:
##df_count_by_ZipCode.describe(include='all')
def Outlier_Finding_IQR(sourcedf):
  print ("\n Finding Outliers using IQR scoring techinique\n")
  Q1 = sourcedf.quantile(0.25)
  Q3 = sourcedf.quantile(0.75)
  IQR = Q3 - Q1
  df_after_clearning_step2 = sourcedf[~((sourcedf < (Q1 - 1.5 * IQR)) |(sourcedf > (Q3 + 1.5 * IQR))).any(axis=1)]
  print ( "\n Before {0} and after{1} clearning the outliers \n".format( sourcedf.shape, df_after_clearning_step2.shape ) )
  print ( "\n Before \n {0} \n after \n{1} \n  \n".format( sourcedf.describe(include='all'), df_after_clearning_step2.describe(include='all') ) ) 

##Outlier_Finding_IQR ( df_house)
##Outlier_Finding_IQR ( df_count_by_ZipCode)

In [ ]:
all_zips = df_house.ZipCode.count().astype(int)
positive_zips = df_house.ZipCode[df_house['ZipCode'] > 0].count().astype(int)
missing_zips = df_house.ZipCode[df_house['ZipCode'] < 1].count().astype(int)
print(f'All Zips {all_zips}, Positive {positive_zips} , Missing {missing_zips}')
print(f'Validity {(positive_zips * 100/all_zips):.2f}%')

df = pd.DataFrame({'Zips':['All', 'Positive', 'Missing'], 'Records':[all_zips, positive_zips, missing_zips]})
ax = df.plot.bar(x='Zips', y='Records', rot=0)
ax.set_ylabel('Zips')
ax.plot()

In [ ]:
###Convert SaleDate to actual date type
print ( "Fist check existing type of Date filed : ", df_house.Date.dtype )

# Check first if we will have wrong conversions
print('Nulls in conversion', pd.to_datetime(df_house['Date']).isnull().sum())
print('NaN in conversion', pd.to_datetime(df_house['Date']).isna().sum())

# Convert and rename column
if pd.to_datetime(df_house['Date']).isna().sum() == 0 and pd.to_datetime(df_house['Date']).isnull().sum() == 0:
    df_house['SaleDate'] = pd.to_datetime(df_house['Date'])
    print('conversion of strings to datetime was successful')
df_house.SaleDate.describe()
print ( "Check New field data type : ", df_house.SaleDate.dtype )
df_house.drop(columns=['Date'])

In [ ]:
##Do a barplot of houses sold per year
df_house['Sale_Year'] = pd.DatetimeIndex(df_house['SaleDate']).year
df_house['Sale_Month'] = pd.DatetimeIndex(df_house['SaleDate']).month

sns.countplot(x=df_house.Sale_Year)
plt.title("Sales by Year")

In [ ]:
# What percentage of data is clean?
def calculate_missing_values_percent_by_columns(diff):
  for col in diff.columns:
    pct_missing = np.mean(diff[col].isnull())
    pct_missing = round(pct_missing*100, 3)
    pct_clean = 100 - pct_missing
    
    print('{0: >20} - pecent clean: {1: >7}% - percent missing: {2: >7}%'.format(col,pct_clean,  pct_missing))
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 *( df.isnull().sum() / len(df) )
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = { 0 : 'Zero Values',1 : 'Missing Values', 2 : '% of Missing Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * ( mz_table['Total Zero Missing Values'] / len(df) )
        mz_table['Data Type'] = df.dtypes
        mz_table['Data Length'] = len(df)
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Missing Values', ascending=False).round(3)

        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table


In [ ]:
#What percentage of data is clen?
print ( 'Percent of Cleanness in the data - Method1')
missing_zero_values_table(df_house)

In [ ]:
print ( 'Percent of Cleanness in the data - Method2')

calculate_missing_values_percent_by_columns(df_house )
